In [1]:
import socket
import sys
import requests
import requests_oauthlib
import json

In [2]:
# Replace the values below with yours
"""
CONSUMER_KEY = 'GpptFmNR9usFQ9AlxXenjH33C'
CONSUMER_SECRET = 'pBVjChbX97GpLpAQG3zm7HTPP34Ui5EbZfOUmCklr0f73WgDWH'
ACCESS_TOKEN = '715011126598078464-V72WAPYEI0m1UwjILwpYyM4EWIvFm3U'
ACCESS_TOKEN_SECRET = 'WCIFP0Bolszk6y7GCPLJ2rZ8BsK1OdbyMWdC51ezNK8xx'
"""

my_auth = requests_oauthlib.OAuth1(CONSUMER_KEY, CONSUMER_SECRET, ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

In [3]:
hashtags = ["Apple", "nsfw", "STAR"]

In [4]:
def get_tweets():
    url = 'https://stream.twitter.com/1.1/statuses/filter.json'
    query_data = [('language', 'en'), ('locations', '-130,-20,100,50'),('track','#')]
    query_url = url + '?' + '&'.join([str(t[0]) + '=' + str(t[1]) for t in query_data])
    response = requests.get(query_url, auth=my_auth, stream=True)
    print(query_url, response)
    return response

In [ ]:
def common(x, y):
    for i in x:
        if i in y:
            return True
    return False
        
def send_tweets_to_spark(http_resp, tcp_connection):
    for line in http_resp.iter_lines():
        try:
            full_tweet = json.loads(line)
            tweet_text = full_tweet['text']
            
            hashtags_raw = full_tweet["entities"]["hashtags"]
            hashtags_got = list()
            for hashtag in hashtags_raw:
                hashtags_got.append(hashtag["text"].lower())
            if hashtags_got:
                print(hashtags_got)
                
            # if any hashtag match happens
            if not common(hashtags, hashtags_got):
                continue
            
            if "extended_entities" in full_tweet:
                print("Tweet Text: ", full_tweet)
                if "media" in full_tweet["extended_entities"]:
                    tweet_image = full_tweet["extended_entities"]["media"][0]["media_url"]
                    print("Tweet url: ", tweet_image)
                    print ("------------------------------------------")
                    tcp_connection.send((full_tweet + '\n').encode('utf-8'))
        except:
            raise
            e = sys.exc_info()[0]
            print("Error: ", e)

In [ ]:
TCP_IP = "localhost"
TCP_PORT = 9010
conn = None
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.bind((TCP_IP, TCP_PORT))
s.listen(1)
print("Waiting for TCP connection...")
conn, addr = s.accept()
print("Connected... Starting getting tweets.")
resp = get_tweets()
send_tweets_to_spark(resp, conn)

Waiting for TCP connection...
Connected... Starting getting tweets.
https://stream.twitter.com/1.1/statuses/filter.json?language=en&locations=-130,-20,100,50&track=# <Response [200]>
['nordiques', 'game7', 'stanleycup']
['teamtrump', 'trump']
['madebywe']
['billiondollarloser', 'blowhole']
['maga']
['its', 'been', 'grace', 'graceofgod', 'all', 'while', 'happybirthdaytome', 'birthday']
['sjsharks']
['lightohioblue']
['schwingstatelovesdogs']
['join', 'ditchmitch', 'senate']
['reds', 'latenightreds', 'borntobaseball']
['got', 'starbucks']
['georgewashingtonbridge']
['coleworld']
['b64bus']
['b64bus']
['nyc', 'manhattan', 'restaurant', 'history', 'gold', 'streetstyle', 'streetshot']
['warriors']
['ख़ामोशी', 'मरते', 'लोग', 'रिश्तें', 'हिंदी', 'yqbaba', 'yqdidi', 'yourquote']
['goavsgo']
['2bedroom', 'independenthouse', 'forsale', 'ganganagar', 'meerut', 'residential']
['smirnoff', 'diageorep']
['blinks']
['savage']
['livtot', 'bpl', 'uefachampionsleague2019']
['taxoxygennow']
['b52baseball

In [ ]:
! lsof -i:9009

In [ ]:
#! kill 